In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)

In [2]:
current_date = pd.Timestamp.today().normalize()

In [3]:
df = pd.DataFrame(
    {
        "date": pd.date_range(
            start="2024-12-01",
            end="2025-03-31",
            freq="D",
        )
    }
)

df["fiscal_year"] = df["date"].dt.to_period("Y-FEB")
df["start_of_fiscal_year"] = df["fiscal_year"].dt.start_time
df["end_of_fiscal_year"] = df["fiscal_year"].dt.end_time.dt.normalize()
# Calculate the fiscal year label
df["fiscal_year_label"] = "FY" + df["date"].dt.to_period("Q-FEB").dt.qyear.apply(
    lambda x: str(x - 1) + "-" + str(x)[2:]
)
display(df)

,date,fiscal_year,start_of_fiscal_year,end_of_fiscal_year,fiscal_year_label
0,2024-12-01,2025,2024-03-01,2025-02-28,FY2024-25
1,2024-12-02,2025,2024-03-01,2025-02-28,FY2024-25
2,2024-12-03,2025,2024-03-01,2025-02-28,FY2024-25
3,2024-12-04,2025,2024-03-01,2025-02-28,FY2024-25
4,2024-12-05,2025,2024-03-01,2025-02-28,FY2024-25
...,...,...,...,...,...
116,2025-03-27,2026,2025-03-01,2026-02-28,FY2025-26
117,2025-03-28,2026,2025-03-01,2026-02-28,FY2025-26
118,2025-03-29,2026,2025-03-01,2026-02-28,FY2025-26
119,2025-03-30,2026,2025-03-01,2026-02-28,FY2025-26


In [11]:
class CalendarBuilder:
    def __init__(self):
        self.current_date = pd.Timestamp.today().normalize()

    @property
    def build(self) -> pd.DataFrame:
        df = self._create_base_df()
        df = self._add_standard_fields(df)
        df = self._add_fiscal_fields(df)
        # df = self._add_transaction_status(df)

        return df.convert_dtypes()

    def _create_base_df(self) -> pd.DataFrame:
        return pd.DataFrame(
            {
                "date": pd.date_range(
                    start="2024-12-01",
                    end="2025-03-31",
                    freq="D",
                )
            }
        )

    def _add_standard_fields(self, df: pd.DataFrame) -> pd.DataFrame:
        dt = df["date"].dt
        periods = self._generate_periods(dt, calendar_type="standard")
        periods_current = self._generate_periods(
            self.current_date, calendar_type="standard"
        )
        iso = dt.isocalendar()

        return df.assign(
            year=dt.year,
            month=dt.month,
            month_abbr=dt.strftime("%b"),
            year_month=dt.strftime("%b %Y"),
            year_month_number=dt.year * 12 + dt.month - 1,
            quarter=dt.quarter,
            year_quarter=periods["quarter"].astype(str),
            iso_week=iso.week,
            day_of_week=iso.day,
            day_name=dt.strftime("%a"),
            same_day_last_year=df["date"] - pd.DateOffset(days=364),
            **self._generate_period_ranges(periods),
            **self._generate_offsets(periods, periods_current),
        )

    def _add_fiscal_fields(self, df: pd.DataFrame) -> pd.DataFrame:
        dt = df["date"].dt
        periods = self._generate_periods(dt, calendar_type="fiscal")
        periods_current = self._generate_periods(
            self.current_date, calendar_type="fiscal"
        )
        fiscal_start_month = periods["fiscal_year"].dt.start_time.dt.month

        return df.assign(
            fiscal_year=periods["fiscal_year"].dt.year,
            fiscal_year_label=self._generate_fiscal_year_label(periods),
            fiscal_month=((dt.month - fiscal_start_month) % 12) + 1,
            fiscal_quarter=periods["fiscal_quarter"].dt.quarter,
            fiscal_year_quarter=periods["fiscal_quarter"].astype(str),
            **self._generate_period_ranges(periods),
            **self._generate_offsets(periods, periods_current),
        )

    # def _add_transaction_status(self, df: pd.DataFrame) -> pd.DataFrame:
    #     last_txn = self.date_loader.last_transaction
    #     df["with_transaction"] = False if last_txn is None else df["date"] <= last_txn
    #     return df

    def _generate_periods(self, date_series, calendar_type="standard"):
        config = {
            "standard": {
                "year": "A",
                "quarter": "Q",
                "month": "M",
                "week": "W",
            },
            "fiscal": {
                "fiscal_year": "A-FEB",
                "fiscal_quarter": "Q-FEB",
            },
        }

        periods = {}
        for key, freq in config[calendar_type].items():
            periods[key] = date_series.to_period(freq)

        return periods

    def _generate_period_ranges(self, periods):
        ranges = {}

        for key, period in periods.items():
            ranges[f"start_of_{key}"] = period.dt.start_time
            ranges[f"end_of_{key}"] = period.dt.end_time.dt.normalize()

        return ranges

    def _generate_offsets(self, periods, periods_current):
        offsets = {}

        for key in periods.keys():
            offsets[f"{key}_offset"] = (periods[key] - periods_current[key]).apply(
                lambda x: x.n
            )
        return offsets

    def _generate_fiscal_year_label(self, periods):
        return "FY" + periods["fiscal_year"].dt.qyear.apply(
            lambda x: str(x - 1) + "-" + str(x)[2:]
        )

In [12]:
generator = CalendarBuilder()
df_calendar = generator.build

/tmp/ipykernel_2177/1573879391.py:88: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'Y' instead.
  periods[key] = date_series.to_period(freq)
/tmp/ipykernel_2177/1573879391.py:88: FutureWarning: 'A-FEB' is deprecated and will be removed in a future version, please use 'Y-FEB' instead.
  periods[key] = date_series.to_period(freq)


In [13]:
display(df_calendar)

,date,year,month,month_abbr,year_month,year_month_number,quarter,year_quarter,iso_week,day_of_week,day_name,same_day_last_year,start_of_year,end_of_year,start_of_quarter,end_of_quarter,start_of_month,end_of_month,start_of_week,end_of_week,year_offset,quarter_offset,month_offset,week_offset,fiscal_year,fiscal_month,fiscal_quarter,fiscal_year_quarter,fiscal_year_label,start_of_fiscal_year,end_of_fiscal_year,start_of_fiscal_quarter,end_of_fiscal_quarter,fiscal_year_offset,fiscal_quarter_offset
0,2024-12-01,2024,12,Dec,Dec 2024,24299,4,2024Q4,48,7,Sun,2023-12-03,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-11-25,2024-12-01,-1,-2,-4,-18,2025,10,4,2025Q4,FY2024-25,2024-03-01,2025-02-28,2024-12-01,2025-02-28,-1,-1
1,2024-12-02,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,1,Mon,2023-12-04,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-2,-4,-17,2025,10,4,2025Q4,FY2024-25,2024-03-01,2025-02-28,2024-12-01,2025-02-28,-1,-1
2,2024-12-03,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,2,Tue,2023-12-05,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-2,-4,-17,2025,10,4,2025Q4,FY2024-25,2024-03-01,2025-02-28,2024-12-01,2025-02-28,-1,-1
3,2024-12-04,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,3,Wed,2023-12-06,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-2,-4,-17,2025,10,4,2025Q4,FY2024-25,2024-03-01,2025-02-28,2024-12-01,2025-02-28,-1,-1
4,2024-12-05,2024,12,Dec,Dec 2024,24299,4,2024Q4,49,4,Thu,2023-12-07,2024-01-01,2024-12-31,2024-10-01,2024-12-31,2024-12-01,2024-12-31,2024-12-02,2024-12-08,-1,-2,-4,-17,2025,10,4,2025Q4,FY2024-25,2024-03-01,2025-02-28,2024-12-01,2025-02-28,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2025-03-27,2025,3,Mar,Mar 2025,24302,1,2025Q1,13,4,Thu,2024-03-28,2025-01-01,2025-12-31,2025-01-01,2025-03-31,2025-03-01,2025-03-31,2025-03-24,2025-03-30,0,-1,-1,-1,2026,1,1,2026Q1,FY2025-26,2025-03-01,2026-02-28,2025-03-01,2025-05-31,0,0
117,2025-03-28,2025,3,Mar,Mar 2025,24302,1,2025Q1,13,5,Fri,2024-03-29,2025-01-01,2025-12-31,2025-01-01,2025-03-31,2025-03-01,2025-03-31,2025-03-24,2025-03-30,0,-1,-1,-1,2026,1,1,2026Q1,FY2025-26,2025-03-01,2026-02-28,2025-03-01,2025-05-31,0,0
118,2025-03-29,2025,3,Mar,Mar 2025,24302,1,2025Q1,13,6,Sat,2024-03-30,2025-01-01,2025-12-31,2025-01-01,2025-03-31,2025-03-01,2025-03-31,2025-03-24,2025-03-30,0,-1,-1,-1,2026,1,1,2026Q1,FY2025-26,2025-03-01,2026-02-28,2025-03-01,2025-05-31,0,0
119,2025-03-30,2025,3,Mar,Mar 2025,24302,1,2025Q1,13,7,Sun,2024-03-31,2025-01-01,2025-12-31,2025-01-01,2025-03-31,2025-03-01,2025-03-31,2025-03-24,2025-03-30,0,-1,-1,-1,2026,1,1,2026Q1,FY2025-26,2025-03-01,2026-02-28,2025-03-01,2025-05-31,0,0


In [14]:
df_calendar.to_csv("calendar.csv", index=False)

In [ ]:
def _generate_fiscal_year_label(self, periods):
    fiscal_year = periods["fiscal_year"].dt.qyear
    start_year = fiscal_year - 1
    end_year = fiscal_year.apply(lambda x: str(x)[2:])
    return "FY" + start_year.astype(str) + "-" + end_year

In [ ]:
def _generate_fiscal_year_label(self, periods):
    fiscal_year = periods["fiscal_year"].dt.qyear
    return "FY" + (fiscal_year - 1).astype(str) + "-" + fiscal_year.astype(str).str[2:]